<a href="https://colab.research.google.com/github/nbht98/attack_cat_or_dog_classification/blob/master/params.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !nvidia-smi

Sat Jun 20 00:22:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!git clone https://github.com/tensorflow/privacy

Cloning into 'privacy'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1773 (delta 5), reused 9 (delta 3), pack-reused 1755
Receiving objects: 100% (1773/1773), 593.80 KiB | 2.46 MiB/s, done.
Resolving deltas: 100% (1219/1219), done.


In [4]:
%cd privacy
!pip install -e .

/content/privacy
Obtaining file:///content/privacy
     |████████████████████████████████| 296kB 4.5MB/s 
  Found existing installation: tensorflow-privacy 0.2.2
    Uninstalling tensorflow-privacy-0.2.2:
      Successfully uninstalled tensorflow-privacy-0.2.2
  Running setup.py develop for tensorflow-privacy


In [5]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf

import numpy as np

tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)

TensorFlow 1.x selected.


In [6]:
!pip install tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

In [7]:
def get_data():
    """Prepare MNIST data."""
    train, test = tf.keras.datasets.mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
    test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    assert train_data.min() == 0.
    assert train_data.max() == 1.
    assert test_data.min() == 0.
    assert test_data.max() == 1.
    return (train_data, train_labels), (test_data, test_labels)
# Get MNIST dataset
(X_train, y_train), (X_test, y_test) = get_data()
    

11493376/11490434 [==============================] - 0s 0us/step


In [59]:
epochs = 10
batch_size = 200

l2_norm_clip = 1
noise_multiplier = 1
num_microbatches = 200
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model = target_model_fn()
target_model.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 2.2075 - acc: 0.3780 - val_loss: 2.0154 - val_acc: 0.5877
Epoch 2/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.8780 - acc: 0.6869 - val_loss: 1.7718 - val_acc: 0.7613
Epoch 3/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.7229 - acc: 0.7904 - val_loss: 1.6706 - val_acc: 0.8299
Epoch 4/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.6535 - acc: 0.8390 - val_loss: 1.6294 - val_acc: 0.8545
Epoch 5/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.6185 - acc: 0.8635 - val_loss: 1.5984 - val_acc: 0.8794
Epoch 6/10
60000/60000 [==============================] - 70s 1ms/sample - loss: 1.5972 - acc: 0.8779 - val_loss: 1.5819 - val_acc: 0.8914
Epoch 7/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5835 - acc: 0.8886 - val_loss: 1.

In [121]:
target_model.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=200)
target_model.save('weights_mnist_batch_200_15.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5580 - acc: 0.9085 - val_loss: 1.5480 - val_acc: 0.9188
Epoch 2/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5540 - acc: 0.9117 - val_loss: 1.5451 - val_acc: 0.9207
Epoch 3/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5505 - acc: 0.9146 - val_loss: 1.5428 - val_acc: 0.9206
Epoch 4/5
60000/60000 [==============================] - 70s 1ms/sample - loss: 1.5479 - acc: 0.9168 - val_loss: 1.5399 - val_acc: 0.9250
Epoch 5/5
60000/60000 [==============================] - 70s 1ms/sample - loss: 1.5457 - acc: 0.9193 - val_loss: 1.5375 - val_acc: 0.9267


In [60]:
epochs = 10
batch_size = 100

l2_norm_clip = 1
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn600():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model600 = target_model_fn600()
target_model600.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 2.0160 - acc: 0.5573 - val_loss: 1.7097 - val_acc: 0.8094
Epoch 2/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.6569 - acc: 0.8338 - val_loss: 1.6041 - val_acc: 0.8731
Epoch 3/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5971 - acc: 0.8768 - val_loss: 1.5725 - val_acc: 0.8984
Epoch 4/10
60000/60000 [==============================] - 70s 1ms/sample - loss: 1.5746 - acc: 0.8951 - val_loss: 1.5597 - val_acc: 0.9070
Epoch 5/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5634 - acc: 0.9032 - val_loss: 1.5530 - val_acc: 0.9125
Epoch 6/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5577 - acc: 0.9074 - val_loss: 1.5460 - val_acc: 0.9184
Epoch 7/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5519 - acc: 0.9130 - val_loss: 1.

In [122]:
target_model600.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model600.save('weights_mnist_batch_100_15.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5324 - acc: 0.9298 - val_loss: 1.5274 - val_acc: 0.9354
Epoch 2/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5340 - acc: 0.9278 - val_loss: 1.5271 - val_acc: 0.9347
Epoch 3/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5332 - acc: 0.9288 - val_loss: 1.5254 - val_acc: 0.9366
Epoch 4/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5322 - acc: 0.9304 - val_loss: 1.5268 - val_acc: 0.9352
Epoch 5/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5308 - acc: 0.9313 - val_loss: 1.5254 - val_acc: 0.9359


In [ ]:
epochs = 10
batch_size = 100

l2_norm_clip = 1
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn600():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model600 = target_model_fn600()
target_model600.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

In [123]:
target_model1200.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=50)
target_model1200.save('weights_mnist_batch_50_15.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 76s 1ms/sample - loss: 1.6058 - acc: 0.8550 - val_loss: 1.5972 - val_acc: 0.8634
Epoch 2/5
60000/60000 [==============================] - 76s 1ms/sample - loss: 1.6099 - acc: 0.8508 - val_loss: 1.6079 - val_acc: 0.8530
Epoch 3/5
60000/60000 [==============================] - 76s 1ms/sample - loss: 1.6143 - acc: 0.8462 - val_loss: 1.6151 - val_acc: 0.8448
Epoch 4/5
60000/60000 [==============================] - 77s 1ms/sample - loss: 1.6155 - acc: 0.8451 - val_loss: 1.6309 - val_acc: 0.8301
Epoch 5/5
60000/60000 [==============================] - 76s 1ms/sample - loss: 1.6259 - acc: 0.8345 - val_loss: 1.6232 - val_acc: 0.8367


In [61]:
epochs = 10
batch_size = 50

l2_norm_clip = 1
noise_multiplier = 1
num_microbatches = 50
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn1200():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model1200 = target_model_fn1200()
target_model1200.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 78s 1ms/sample - loss: 1.8524 - acc: 0.6826 - val_loss: 1.6399 - val_acc: 0.8400
Epoch 2/10
60000/60000 [==============================] - 76s 1ms/sample - loss: 1.6193 - acc: 0.8521 - val_loss: 1.6040 - val_acc: 0.8644
Epoch 3/10
60000/60000 [==============================] - 76s 1ms/sample - loss: 1.5974 - acc: 0.8680 - val_loss: 1.5818 - val_acc: 0.8805
Epoch 4/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 1.5902 - acc: 0.8726 - val_loss: 1.5829 - val_acc: 0.8801
Epoch 5/10
60000/60000 [==============================] - 76s 1ms/sample - loss: 1.5945 - acc: 0.8669 - val_loss: 1.5957 - val_acc: 0.8655
Epoch 6/10
60000/60000 [==============================] - 77s 1ms/sample - loss: 1.6012 - acc: 0.8608 - val_loss: 1.5993 - val_acc: 0.8616
Epoch 7/10
60000/60000 [==============================] - 76s 1ms/sample - loss: 1.6006 - acc: 0.8611 - val_loss: 1.

In [62]:
epochs = 10
batch_size = 100

l2_norm_clip = 2
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn2_clip():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model2_clip = target_model_fn2_clip()
target_model2_clip.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.8960 - acc: 0.6380 - val_loss: 1.5990 - val_acc: 0.8893
Epoch 2/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5817 - acc: 0.8936 - val_loss: 1.5521 - val_acc: 0.9157
Epoch 3/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5549 - acc: 0.9131 - val_loss: 1.5418 - val_acc: 0.9241
Epoch 4/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5461 - acc: 0.9187 - val_loss: 1.5406 - val_acc: 0.9236
Epoch 5/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5418 - acc: 0.9216 - val_loss: 1.5334 - val_acc: 0.9296
Epoch 6/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5412 - acc: 0.9221 - val_loss: 1.5306 - val_acc: 0.9344
Epoch 7/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5404 - acc: 0.9222 - val_loss: 1.

In [125]:
target_model2_clip.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model2_clip.save('weights_mnist_clip_2_15.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5445 - acc: 0.9169 - val_loss: 1.5329 - val_acc: 0.9283
Epoch 2/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5467 - acc: 0.9147 - val_loss: 1.5416 - val_acc: 0.9200
Epoch 3/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5488 - acc: 0.9122 - val_loss: 1.5424 - val_acc: 0.9199
Epoch 4/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5540 - acc: 0.9069 - val_loss: 1.5554 - val_acc: 0.9060
Epoch 5/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5579 - acc: 0.9031 - val_loss: 1.5508 - val_acc: 0.9095


In [63]:
epochs = 10
batch_size = 100

l2_norm_clip = 3
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn3_clip():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model3_clip = target_model_fn3_clip()
target_model3_clip.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 1.9004 - acc: 0.5951 - val_loss: 1.6827 - val_acc: 0.7903
Epoch 2/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6613 - acc: 0.8079 - val_loss: 1.6488 - val_acc: 0.8168
Epoch 3/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6457 - acc: 0.8185 - val_loss: 1.6360 - val_acc: 0.8263
Epoch 4/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6432 - acc: 0.8185 - val_loss: 1.6330 - val_acc: 0.8279
Epoch 5/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6453 - acc: 0.8162 - val_loss: 1.6452 - val_acc: 0.8157
Epoch 6/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6465 - acc: 0.8147 - val_loss: 1.6397 - val_acc: 0.8204
Epoch 7/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.6521 - acc: 0.8085 - val_loss: 1.

In [ ]:
target_model3_clip.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model3_clip.save('weights_mnist_clip_3_15.h5')

In [64]:
epochs = 10
batch_size = 100

l2_norm_clip = 4
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn4_clip():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model4_clip = target_model_fn4_clip()
target_model4_clip.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 1.9682 - acc: 0.5062 - val_loss: 1.7350 - val_acc: 0.7349
Epoch 2/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.7287 - acc: 0.7351 - val_loss: 1.7142 - val_acc: 0.7479
Epoch 3/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.7294 - acc: 0.7318 - val_loss: 1.7264 - val_acc: 0.7334
Epoch 4/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.7362 - acc: 0.7242 - val_loss: 1.7400 - val_acc: 0.7206
Epoch 5/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.7429 - acc: 0.7169 - val_loss: 1.7456 - val_acc: 0.7137
Epoch 6/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.7444 - acc: 0.7152 - val_loss: 1.7338 - val_acc: 0.7265
Epoch 7/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.7468 - acc: 0.7132 - val_loss: 1.

In [ ]:
target_model4_clip.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model4_clip.save('weights_mnist_clip_4_15.h5')

In [65]:
epochs = 10
batch_size = 100

l2_norm_clip = 5
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn5_clip():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model5_clip = target_model_fn5_clip()
target_model5_clip.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.8886 - acc: 0.5908 - val_loss: 1.7588 - val_acc: 0.7057
Epoch 2/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.7745 - acc: 0.6867 - val_loss: 1.7744 - val_acc: 0.6875
Epoch 3/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.7889 - acc: 0.6711 - val_loss: 1.7844 - val_acc: 0.6753
Epoch 4/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.7978 - acc: 0.6618 - val_loss: 1.7849 - val_acc: 0.6744
Epoch 5/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.7981 - acc: 0.6618 - val_loss: 1.7942 - val_acc: 0.6660
Epoch 6/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.8010 - acc: 0.6591 - val_loss: 1.8045 - val_acc: 0.6556
Epoch 7/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.8190 - acc: 0.6410 - val_loss: 1.

In [ ]:
target_model5_clip.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model5_clip.save('weights_mnist_clip_5_15.h5')

In [113]:
epochs = 10
batch_size = 100

l2_norm_clip = 0.5
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn05_clip():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model05_clip = target_model_fn05_clip()
target_model05_clip.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)
target_model05_clip.save('weights_mnist_10_005_01_100_01.h5')


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 2.1303 - acc: 0.4502 - val_loss: 1.8913 - val_acc: 0.6560
Epoch 2/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.7724 - acc: 0.7600 - val_loss: 1.6861 - val_acc: 0.8213
Epoch 3/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.6556 - acc: 0.8401 - val_loss: 1.6202 - val_acc: 0.8641
Epoch 4/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6103 - acc: 0.8699 - val_loss: 1.5916 - val_acc: 0.8851
Epoch 5/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5882 - acc: 0.8854 - val_loss: 1.5785 - val_acc: 0.8921
Epoch 6/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5771 - acc: 0.8934 - val_loss: 1.5689 - val_acc: 0.9000
Epoch 7/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5689 - acc: 0.8998 - val_loss: 1.

In [127]:
target_model05_clip.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model05_clip.save('weights_mnist_clip_05_15.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5506 - acc: 0.9149 - val_loss: 1.5473 - val_acc: 0.9152
Epoch 2/5
60000/60000 [==============================] - 74s 1ms/sample - loss: 1.5478 - acc: 0.9167 - val_loss: 1.5436 - val_acc: 0.9205
Epoch 3/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5450 - acc: 0.9197 - val_loss: 1.5418 - val_acc: 0.9225
Epoch 4/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5417 - acc: 0.9223 - val_loss: 1.5378 - val_acc: 0.9260
Epoch 5/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5388 - acc: 0.9248 - val_loss: 1.5342 - val_acc: 0.9297


In [128]:
target_model15_clip.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model15_clip.save('weights_mnist_clip_15_15.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5434 - acc: 0.9190 - val_loss: 1.5342 - val_acc: 0.9295
Epoch 2/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5441 - acc: 0.9179 - val_loss: 1.5356 - val_acc: 0.9262
Epoch 3/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5407 - acc: 0.9215 - val_loss: 1.5323 - val_acc: 0.9306
Epoch 4/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5389 - acc: 0.9228 - val_loss: 1.5360 - val_acc: 0.9261
Epoch 5/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5390 - acc: 0.9228 - val_loss: 1.5350 - val_acc: 0.9263


In [68]:
epochs = 10
batch_size = 100

l2_norm_clip = 1
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.05

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn005_lr():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model005_lr = target_model_fn005_lr()
target_model005_lr.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 2.2016 - acc: 0.3956 - val_loss: 1.9763 - val_acc: 0.6370
Epoch 2/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.8436 - acc: 0.7175 - val_loss: 1.7363 - val_acc: 0.7845
Epoch 3/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 1.6966 - acc: 0.8134 - val_loss: 1.6499 - val_acc: 0.8466
Epoch 4/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.6364 - acc: 0.8506 - val_loss: 1.6107 - val_acc: 0.8675
Epoch 5/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.6062 - acc: 0.8712 - val_loss: 1.5869 - val_acc: 0.8876
Epoch 6/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5908 - acc: 0.8813 - val_loss: 1.5772 - val_acc: 0.8940
Epoch 7/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5813 - acc: 0.8895 - val_loss: 1.

In [71]:
epochs = 10
batch_size = 100

l2_norm_clip = 1
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.15

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn015_lr():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model015_lr = target_model_fn015_lr()
target_model015_lr.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 1.9590 - acc: 0.6019 - val_loss: 1.6866 - val_acc: 0.8134
Epoch 2/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6354 - acc: 0.8468 - val_loss: 1.5989 - val_acc: 0.8751
Epoch 3/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5880 - acc: 0.8812 - val_loss: 1.5708 - val_acc: 0.8963
Epoch 4/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5698 - acc: 0.8965 - val_loss: 1.5538 - val_acc: 0.9115
Epoch 5/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5577 - acc: 0.9071 - val_loss: 1.5477 - val_acc: 0.9163
Epoch 6/10
60000/60000 [==============================] - 77s 1ms/sample - loss: 1.5515 - acc: 0.9129 - val_loss: 1.5392 - val_acc: 0.9250
Epoch 7/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5478 - acc: 0.9158 - val_loss: 1.

In [76]:
epochs = 10
batch_size = 100

l2_norm_clip = 1
noise_multiplier = 0.5
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn05_n():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model05_n = target_model_fn05_n()
target_model05_n.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 1.9687 - acc: 0.6263 - val_loss: 1.6829 - val_acc: 0.8272
Epoch 2/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 1.6268 - acc: 0.8648 - val_loss: 1.5831 - val_acc: 0.8927
Epoch 3/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5795 - acc: 0.8932 - val_loss: 1.5627 - val_acc: 0.9039
Epoch 4/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5628 - acc: 0.9054 - val_loss: 1.5496 - val_acc: 0.9165
Epoch 5/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5537 - acc: 0.9123 - val_loss: 1.5435 - val_acc: 0.9222
Epoch 6/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5480 - acc: 0.9174 - val_loss: 1.5380 - val_acc: 0.9268
Epoch 7/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5440 - acc: 0.9204 - val_loss: 1.

In [80]:
epochs = 10
batch_size = 100

l2_norm_clip = 1
noise_multiplier = 2
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn2_n():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model2_n = target_model_fn2_n()
target_model2_n.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 2.0173 - acc: 0.5480 - val_loss: 1.7140 - val_acc: 0.8120
Epoch 2/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6581 - acc: 0.8338 - val_loss: 1.6104 - val_acc: 0.8648
Epoch 3/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5976 - acc: 0.8737 - val_loss: 1.5731 - val_acc: 0.8978
Epoch 4/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5807 - acc: 0.8881 - val_loss: 1.5710 - val_acc: 0.8976
Epoch 5/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5734 - acc: 0.8928 - val_loss: 1.5644 - val_acc: 0.9003
Epoch 6/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5723 - acc: 0.8918 - val_loss: 1.5641 - val_acc: 0.9000
Epoch 7/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5693 - acc: 0.8941 - val_loss: 1.

In [81]:
epochs = 10
batch_size = 100

l2_norm_clip = 1
noise_multiplier = 1.5
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn15_n():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model15_n = target_model_fn15_n()
target_model15_n.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 2.0241 - acc: 0.5630 - val_loss: 1.7252 - val_acc: 0.8099
Epoch 2/10
60000/60000 [==============================] - 70s 1ms/sample - loss: 1.6451 - acc: 0.8535 - val_loss: 1.5938 - val_acc: 0.8850
Epoch 3/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5870 - acc: 0.8867 - val_loss: 1.5729 - val_acc: 0.8974
Epoch 4/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5713 - acc: 0.8976 - val_loss: 1.5583 - val_acc: 0.9102
Epoch 5/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5600 - acc: 0.9060 - val_loss: 1.5514 - val_acc: 0.9145
Epoch 6/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5551 - acc: 0.9098 - val_loss: 1.5461 - val_acc: 0.9185
Epoch 7/10
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5496 - acc: 0.9151 - val_loss: 1.

In [133]:
target_model15_n.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model15_n.save('weights_mnist_noise_15_15.h5')


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5427 - acc: 0.9200 - val_loss: 1.5366 - val_acc: 0.9262
Epoch 2/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5454 - acc: 0.9168 - val_loss: 1.5453 - val_acc: 0.9184
Epoch 3/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5471 - acc: 0.9150 - val_loss: 1.5404 - val_acc: 0.9218
Epoch 4/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5486 - acc: 0.9136 - val_loss: 1.5432 - val_acc: 0.9191
Epoch 5/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5460 - acc: 0.9158 - val_loss: 1.5394 - val_acc: 0.9231


In [134]:
target_model05_n.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model05_n.save('weights_mnist_noise_05_15.h5')


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5310 - acc: 0.9323 - val_loss: 1.5225 - val_acc: 0.9413
Epoch 2/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5287 - acc: 0.9347 - val_loss: 1.5202 - val_acc: 0.9433
Epoch 3/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5261 - acc: 0.9371 - val_loss: 1.5172 - val_acc: 0.9461
Epoch 4/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5246 - acc: 0.9383 - val_loss: 1.5154 - val_acc: 0.9475
Epoch 5/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5226 - acc: 0.9398 - val_loss: 1.5147 - val_acc: 0.9481


In [135]:
target_model2_n.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model2_n.save('weights_mnist_noise_2_15.h5')


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5659 - acc: 0.8957 - val_loss: 1.5565 - val_acc: 0.9055
Epoch 2/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5737 - acc: 0.8880 - val_loss: 1.5760 - val_acc: 0.8858
Epoch 3/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5808 - acc: 0.8801 - val_loss: 1.5713 - val_acc: 0.8884
Epoch 4/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5802 - acc: 0.8811 - val_loss: 1.5775 - val_acc: 0.8847
Epoch 5/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5891 - acc: 0.8716 - val_loss: 1.5848 - val_acc: 0.8756


In [82]:
epochs = 10
batch_size = 100

l2_norm_clip = 1.5
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

def target_model_fn15_clip():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_norm_clip,
      noise_multiplier=noise_multiplier,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

target_model15_clip = target_model_fn15_clip()
target_model15_clip.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 75s 1ms/sample - loss: 1.9476 - acc: 0.6159 - val_loss: 1.6419 - val_acc: 0.8579
Epoch 2/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.6060 - acc: 0.8760 - val_loss: 1.5633 - val_acc: 0.9107
Epoch 3/10
60000/60000 [==============================] - 74s 1ms/sample - loss: 1.5644 - acc: 0.9058 - val_loss: 1.5444 - val_acc: 0.9257
Epoch 4/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5501 - acc: 0.9160 - val_loss: 1.5411 - val_acc: 0.9244
Epoch 5/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5477 - acc: 0.9177 - val_loss: 1.5350 - val_acc: 0.9291
Epoch 6/10
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5424 - acc: 0.9219 - val_loss: 1.5340 - val_acc: 0.9293
Epoch 7/10
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5423 - acc: 0.9205 - val_loss: 1.

Change Learning Rate

In [83]:
target_model005_lr.fit(X_train, y_train,
          epochs=15,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 75s 1ms/sample - loss: 2.1233 - acc: 0.5190 - val_loss: 1.8746 - val_acc: 0.7080
Epoch 2/15
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.7676 - acc: 0.7712 - val_loss: 1.6931 - val_acc: 0.8120
Epoch 3/15
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6658 - acc: 0.8292 - val_loss: 1.6312 - val_acc: 0.8564
Epoch 4/15
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6233 - acc: 0.8595 - val_loss: 1.6044 - val_acc: 0.8747
Epoch 5/15
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.6011 - acc: 0.8757 - val_loss: 1.5853 - val_acc: 0.8866
Epoch 6/15
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5866 - acc: 0.8853 - val_loss: 1.5719 - val_acc: 0.8987
Epoch 7/15
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5771 - acc: 0.8922 - val_loss: 1.

In [ ]:
target_model005_lr

In [84]:
target_model600.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5395 - acc: 0.9235 - val_loss: 1.5320 - val_acc: 0.9304
Epoch 2/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5374 - acc: 0.9254 - val_loss: 1.5299 - val_acc: 0.9325
Epoch 3/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5355 - acc: 0.9268 - val_loss: 1.5280 - val_acc: 0.9350
Epoch 4/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5351 - acc: 0.9271 - val_loss: 1.5253 - val_acc: 0.9372
Epoch 5/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5333 - acc: 0.9292 - val_loss: 1.5253 - val_acc: 0.9372


In [131]:
target_model015_lr.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=100)
target_model015_lr.save('weights_mnist_lr_15_15.h5')


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5455 - acc: 0.9158 - val_loss: 1.5386 - val_acc: 0.9223
Epoch 2/5
60000/60000 [==============================] - 73s 1ms/sample - loss: 1.5470 - acc: 0.9143 - val_loss: 1.5452 - val_acc: 0.9177
Epoch 3/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5479 - acc: 0.9134 - val_loss: 1.5457 - val_acc: 0.9162
Epoch 4/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5477 - acc: 0.9134 - val_loss: 1.5413 - val_acc: 0.9204
Epoch 5/5
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.5451 - acc: 0.9162 - val_loss: 1.5424 - val_acc: 0.9196


In [132]:
target_model005_lr.save('weights_mnist_lr_05_15.h5')

In [ ]:
l2_norm_clip = 1
noise_multiplier = 1
num_microbatches = 100
learning_rate = 0.15


In [87]:
from google.colab import files

In [111]:
target_model1200.save('weights_mnist_10_01_01_50_01.h5')

In [30]:
# Test the success of the attack.

# Prepare examples that were in the training, and out of the training.
data_in = X_train[:4000], y_train[:4000]
data_out = X_test[:4000], y_test[:4000]

# Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

# Compute the attack accuracy.
attack_guesses = amb.predict(attack_test_data)
attack_accuracy = np.mean(attack_guesses == real_membership_labels)

print(attack_accuracy)

0.513375


In [119]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=60000, batch_size=2000, noise_multiplier=3, epochs=10, delta=1e-5)

DP-SGD with sampling rate = 3.33% and noise_multiplier = 3 iterated over 300 steps satisfies differential privacy with eps = 1.02 and delta = 1e-05.
The optimal RDP order is 23.0.


(1.0156246807062732, 23.0)